In [11]:
#!/usr/bin/python
import sys
import os
import subprocess as sp
import io
import networkx as nx
from importlib import reload

root_path = sp.run(['git', 'rev-parse', '--show-toplevel'], stdout=sp.PIPE).stdout.decode('utf-8')[:-1]
sys.path.append(os.path.join(root_path, 'source'))
sys.path.append(os.path.join(root_path, 'source', 'matlab_libs'))

import my_utils as my
from geometry.vertices_graph import vertices_graph
from data_preparation.extract_and_triangulate_lib import *

class IpyExit(SystemExit):
    """Exit Exception for IPython.

    Exception temporarily redirects stderr to buffer.
    """
    def __init__(self):
        # print("exiting")  # optionally print some message to stdout, too
        # ... or do other stuff before exit
        sys.stderr = io.StringIO()

    def __del__(self):
        sys.stderr.close()
        sys.stderr = sys.__stderr__  # restore from backup

In [12]:
args = ['1Z0K_uR_1000.pdb', 'A', '1Z0K_C_36457.pdb', 'A', '2.0']

ground_truth_cut_dist = float(args[4]) if len(args) > 4 else 2.0
to_reload = (args[5] == '1') if (len(args) > 5) else True
u_pdb_filename, u_chain_name, u_pdb_filepath, u_chain_filepath_base, u_chain_filepath = \
    parse_names(args[0:2], tmp_dir=masif_opts["tmp_dir"])

if(to_reload):
    # Extract chains of interest.
    extractPDB(u_pdb_filepath, u_chain_filepath, u_chain_name)

# construct the mesh.
u_regular_mesh, u_vertex_normals, u_vertices, u_names = \
    msms_wrap(u_chain_filepath)
u_vertex_hbond, u_vertex_hphobicity, u_vertex_charges = \
    compute_features(u_chain_filepath_base, u_vertices, u_names, u_regular_mesh)

ply_filepath = u_chain_filepath_base + '_d' + str(ground_truth_cut_dist) + '.ply'
if 'compute_iface' in masif_opts and masif_opts['compute_iface']:
    C_pdb_filename, C_chain_name, C_pdb_filepath, C_chain_filepath_base, C_chain_filepath = \
        parse_names(args[2:4])
    
    C_regular_mesh, C_vertex_normals, C_vertices, C_names = \
        msms_wrap(C_pdb_filepath)
    
    #iface = find_iface(C_regular_mesh, u_regular_mesh, ground_truth_cut_dist)
    kdt = KDTree(C_regular_mesh.vertices)
    d, r = kdt.query(u_regular_mesh.vertices)
    d = np.square(d) # Square d, because this is how it was in the pyflann version.
    assert(len(d) == len(u_regular_mesh.vertices))
    iface_v = np.where(d >= ground_truth_cut_dist)[0]

    iface = np.zeros(len(u_regular_mesh.vertices))
    iface[iface_v] = 1.0
    G, edges = vertices_graph(u_regular_mesh, weighted=False)
    
    save_ply(ply_filepath, u_regular_mesh.vertices,\
                        u_regular_mesh.faces, normals=u_vertex_normals, charges=u_vertex_charges,\
                        normalize_charges=True, hbond=u_vertex_hbond, hphob=u_vertex_hphobicity,\
                        iface=iface)
else:
    save_ply(ply_filepath, u_regular_mesh.vertices,\
                        u_regular_mesh.faces, normals=u_vertex_normals, charges=u_vertex_charges,\
                        normalize_charges=True, hbond=u_vertex_hbond, hphob=u_vertex_hphobicity)
    
copy_tmp2dst(ply_filepath, masif_opts['ply_chain_dir'])
copy_tmp2dst(u_chain_filepath, masif_opts['pdb_chain_dir'])

Removing degenerated triangles


/home/student/anaconda3/envs/py36_masif/lib/python3.6/site-packages/Bio/PDB/Atom.py:205: PDBConstructionWarning: Could not assign element 'O1G' for Atom (name=O1G) with given element 'VS'
  warnings.warn(msg, PDBConstructionWarning)
/home/student/anaconda3/envs/py36_masif/lib/python3.6/site-packages/Bio/PDB/Atom.py:205: PDBConstructionWarning: Could not assign element 'O2G' for Atom (name=O2G) with given element 'VS'
  warnings.warn(msg, PDBConstructionWarning)
/home/student/anaconda3/envs/py36_masif/lib/python3.6/site-packages/Bio/PDB/Atom.py:205: PDBConstructionWarning: Could not assign element 'O3G' for Atom (name=O3G) with given element 'VS'
  warnings.warn(msg, PDBConstructionWarning)


Removing degenerated triangles
data_preparation/01-benchmark_surfaces/1Z0K_uR_1000_A_d2.0.ply written
data_preparation/01-benchmark_pdbs/1Z0K_uR_1000_A.pdb written


In [43]:
import geometry.vertices_graph
geometry.vertices_graph = reload(geometry.vertices_graph)

G = geometry.vertices_graph.vertices_graph(u_regular_mesh, weighted=False)
N_verts = len(u_regular_mesh.vertices)
print(nx.is_connected(G))

not_iface_v = []
for v in range(N_verts):
    if(not v in iface_v):
        not_iface_v.append(v)
not_iface_v = np.array(not_iface_v)

G.remove_nodes_from(not_iface_v)
print(nx.is_connected(G))
G_comps = [np.array(list(c)) for c in nx.connected_components(G)]
t_iface = []
for Gc in G_comps:
    G_comps_len = len(Gc)
    if(G_comps_len > 20):
        t_iface.append(Gc)
        
t_iface = [max(G_comps, key=len)]
t_iface = np.concatenate(t_iface)
print(t_iface)
#print(nx.is_connected(G0))

True
False
[2561 1538    3 2564 2565    5 1033 1037 2574 2575 3086 1041  533 1559
 1047   25   26 1051 2077 3103 1568 2592 2597 1575 1065  555 3117 3119
  563 1589 2614 1080 3129 3130 3131   61 3134  574   64 1088  578 2114
 1601 2626  582 1089 1605 1099  592 1110 1624  600   90   91 3164 1628
 3167 2657 2146 2147 1635  618 2154  621 1647  624  625  628 2164  631
 1656 1657 1655 3191 1144 2685 2174 2177  642 1155 2180 3206 1159 1160
 2694 2698  139 1680 1169 2194 2707 2708  146 1686 3222 2711 1176  147
 2203 2716  660  152 3232 1696  673 1185 1701 1190 1703 1704 2731 1198
 1200  689 2224 1715  180 2740 2230 2231 1717 3260 2752 2753 3264  707
 2245 2758 3275 2764 2765 1233 2774 1241 1755 2268 3294 1250  738 2279
 1769 3305 1771 1260 2287 2297 2810 2811 3324 1277 3326 1791 1280 1796
  260 3334 2310  264  267 3340 1804 2831  784  273 3349 1302 1303  792
 1817 1304 1305 2844  284  288 2338 2851 1315 1828  293  802 2852  811
 3372 2349 2860 1840 3377  309 1845 2871 3383 3386 3387 2876 1852 

In [18]:
#list(G.nodes(data=True))

In [13]:
G0 = G

In [18]:
iface_v

array([   3,    5,   14,   18,   25,   26,   42,   43,   45,   61,   64,
         79,   90,   91,   95,  128,  134,  139,  143,  146,  147,  152,
        155,  170,  180,  187,  191,  199,  204,  205,  219,  228,  231,
        232,  236,  239,  245,  248,  260,  264,  266,  267,  273,  280,
        282,  284,  288,  292,  293,  307,  308,  309,  312,  318,  321,
        323,  325,  333,  334,  336,  337,  339,  341,  343,  345,  349,
        357,  363,  364,  371,  381,  384,  392,  408,  409,  410,  424,
        433,  437,  438,  441,  448,  450,  452,  454,  456,  457,  460,
        472,  479,  481,  482,  484,  488,  495,  496,  509,  513,  525,
        527,  528,  533,  548,  554,  555,  557,  563,  567,  571,  573,
        574,  577,  578,  582,  583,  584,  592,  600,  604,  606,  612,
        618,  620,  621,  624,  625,  626,  627,  628,  631,  635,  638,
        640,  642,  649,  658,  659,  660,  668,  673,  685,  689,  696,
        698,  707,  709,  712,  713,  719,  727,  7

In [36]:
#vertices_graph??

Signature: vertices_graph(mesh, weighted=False)
Docstring: <no docstring>
Source:   
def vertices_graph(mesh, weighted=False):
    # Graph 
    G = nx.Graph()
    n = len(mesh.vertices)
    G.add_nodes_from(np.arange(n))

    # Get edges
    f = np.array(mesh.faces, dtype = int)
    rowi = np.concatenate([f[:,0], f[:,0], f[:,1], f[:,1], f[:,2], f[:,2]], axis = 0)
    rowj = np.concatenate([f[:,1], f[:,2], f[:,0], f[:,2], f[:,0], f[:,1]], axis = 0)

    if(weighted):
        #verts = mesh.vertices
        # Get weights 
        edgew = mesh.vertices[rowi] - mesh.vertices[rowj]
        edgew = scipy.linalg.norm(edgew, axis=1)
        wedges = np.stack([rowi, rowj, edgew]).T
        G.add_weighted_edges_from(wedges)
        return G, wedges
    else:
        edges = np.stack([rowi, rowj]).T
        G.add_edges_from(edges)
        return G, edges
File:      ~/masif/source/geometry/vertices_graph.py
Type:      function
